In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import year, to_date, col

spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local") \
    .getOrCreate()


In [2]:
##Get the schemas

import json

file_path = 'data/retail_db/schemas.json'
schemas = json.load(open(file_path))

In [3]:
## Function to get the columnNames

def get_column_names(schemas, dsName):
    column_details = schemas[dsName]
    sorted_column = sorted(column_details, key= lambda col : col['column_position'])
    return [column['column_name'] for column in sorted_column]

In [4]:
## Define Column Schema

customers_column =get_column_names(schemas,'customers')
order_colum = get_column_names(schemas,'orders')
order_item_column = get_column_names(schemas,'order_items')

In [5]:
### Read the sources

##customer
customerDataset = spark.read.option('header','false').csv('data/retail_db/customers/').toDF(*customers_column)
##orders
ordersDataset = spark.read.option('header','false').csv('data/retail_db/orders/').toDF(*order_colum)
#order_items
orderItemDataset = spark.read.option('header','false').csv('data/retail_db/order_items/').toDF(*order_item_column)


In [6]:
ordersDataset = ordersDataset.withColumn("year_date", year(to_date("order_date"))) \
                             .filter(col("year_date") == "2013")


ordersDataset.show()

+--------+--------------------+-----------------+---------------+---------+
|order_id|          order_date|order_customer_id|   order_status|year_date|
+--------+--------------------+-----------------+---------------+---------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|     2013|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|     2013|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|     2013|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|     2013|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|     2013|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|     2013|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|     2013|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|     2013|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|     2013|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT|     2013|
|      11|20